# Dataset


<a href="https://archive.ics.uci.edu/dataset/759/glioma+grading+clinical+and+mutation+features+dataset">Glioma Grading Clinical and Mutation Features</a>


<a href="https://github.com/uci-ml-repo/ucimlrepo/tree/main">github: ucimlrepo package</a>



Citation: Tasci, E., Zhuge, Y., Kaur, H., Camphausen, K., & Krauze, A. V. (2022). Hierarchical Voting-Based Feature Selection and Ensemble Learning Model Scheme for Glioma Grading with Clinical and Molecular Characteristics. International Journal of Molecular Sciences, 23(22), 14155.


# 1. Import

In [115]:
import pandas as pd
import numpy as np
import seaborn as sn

import sqlalchemy # generate tabular data-set; pip install sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text

from ucimlrepo import fetch_ucirepo, list_available_datasets

# 2. Overview Dataset 

In [18]:
# fetch dataset
glioma_feat = fetch_ucirepo(id=759) 

In [9]:
type(glioma_feat)

ucimlrepo.dotdict.dotdict

In [76]:
glioma_feat.keys()

dict_keys(['data', 'metadata', 'variables'])

In [77]:
# Glioma Metadata
glioma_feat['metadata'].keys()

dict_keys(['uci_id', 'name', 'repository_url', 'data_url', 'abstract', 'area', 'tasks', 'characteristics', 'num_instances', 'num_features', 'feature_types', 'demographics', 'target_col', 'index_col', 'has_missing_values', 'missing_values_symbol', 'year_of_dataset_creation', 'last_updated', 'dataset_doi', 'creators', 'intro_paper', 'additional_info'])

In [50]:
# Abstract
glioma_feat.metadata['abstract']

'Gliomas are the most common primary tumors of the brain. They can be graded as LGG (Lower-Grade Glioma) or GBM (Glioblastoma Multiforme) depending on the histological/imaging criteria. Clinical and molecular/mutation factors are also very crucial for the grading process. Molecular tests are expensive to help accurately diagnose glioma patients.    In this dataset, the most frequently mutated 20 genes and 3 clinical features are considered from TCGA-LGG and TCGA-GBM brain glioma projects.  The prediction task is to determine whether a patient is LGG or GBM with a given clinical and molecular/mutation features. The main objective is to find the optimal subset of mutation genes and clinical features for the glioma grading process to improve performance and reduce costs.  '

In [25]:
# access metadata
print(glioma_feat.metadata.uci_id)
print(glioma_feat.metadata.num_instances)
print(glioma_feat.metadata.additional_info.summary)

759
839
None


In [57]:
display(glioma_feat.variables['name'])

0                Grade
1               Gender
2     Age_at_diagnosis
3                 Race
4                 IDH1
5                 TP53
6                 ATRX
7                 PTEN
8                 EGFR
9                  CIC
10               MUC16
11              PIK3CA
12                 NF1
13              PIK3R1
14               FUBP1
15                 RB1
16              NOTCH1
17                BCOR
18               CSMD3
19             SMARCA4
20              GRIN2A
21                IDH2
22                FAT4
23              PDGFRA
Name: name, dtype: object

In [59]:
display(glioma_feat.variables['description'])

0     Glioma grade class information (0 = "LGG"; 1 =...
1                     Gender (0 = "male"; 1 = "female")
2     Age at diagnosis with the calculated number of...
3     Race  (0 = "white";  1 = "black or african Ame...
4     isocitrate dehydrogenase (NADP(+))1 (0 = NOT_M...
5      tumor protein p53 (0 = NOT_MUTATED; 1 = MUTATED)
6     ATRX chromatin remodeler (0 = NOT_MUTATED; 1 =...
7     phosphatase and tensin homolog (0 = NOT_MUTATE...
8     epidermal growth factor receptor (0 = NOT_MUTA...
9     capicua transcriptional repressor (0 = NOT_MUT...
10    mucin 16, cell surface associated (0 = NOT_MUT...
11    phosphatidylinositol-4,5-bisphosphate 3-kinase...
12       neurofibromin 1 (0 = NOT_MUTATED; 1 = MUTATED)
13    phosphoinositide-3-kinase regulatory subunit 1...
14    far upstream element binding protein 1 (0 = NO...
15    RB transcriptional corepressor 1 (0 = NOT_MUTA...
16      notch receptor 1 (0 = NOT_MUTATED; 1 = MUTATED)
17      BCL6 corepressor (0 = NOT_MUTATED; 1 = M

In [67]:
glioma_feat.data.headers

Index(['Case_ID', 'Gender', 'Age_at_diagnosis', 'Race', 'IDH1', 'TP53', 'ATRX',
       'PTEN', 'EGFR', 'CIC', 'MUC16', 'PIK3CA', 'NF1', 'PIK3R1', 'FUBP1',
       'RB1', 'NOTCH1', 'BCOR', 'CSMD3', 'SMARCA4', 'GRIN2A', 'IDH2', 'FAT4',
       'PDGFRA', 'Grade'],
      dtype='object')

In [79]:
glioma_feat.metadata['data_url']

'https://archive.ics.uci.edu/static/public/759/data.csv'

# 3. Load Data

In [83]:
!dir

 Datentr„ger in Laufwerk C: ist Windows
 Volumeseriennummer: 20BD-6958

 Verzeichnis von C:\Users\andre\Documents\IRONHACK\UNIT_4\Mini_Project_SQL\Mini_Project_SQL\notebooks

11.03.2024  21:14    <DIR>          .
11.03.2024  17:55    <DIR>          ..
11.03.2024  17:47    <DIR>          .ipynb_checkpoints
11.03.2024  21:14            38.516 Mini_Project_SQL.ipynb
               1 Datei(en),         38.516 Bytes
               3 Verzeichnis(se), 707.907.223.552 Bytes frei


In [91]:
# data (as pandas dataframes) 
df_target = glioma_feat.data.original

df_info = pd.read_csv("../data/raw/glioma+grading+clinical+and+mutation+features+dataset/TCGA_GBM_LGG_Mutations_all.csv", usecols=['Case_ID', 'Primary_Diagnosis'])

In [92]:
df_target.head()

,Case_ID,Gender,Age_at_diagnosis,Race,IDH1,TP53,ATRX,PTEN,EGFR,CIC,...,RB1,NOTCH1,BCOR,CSMD3,SMARCA4,GRIN2A,IDH2,FAT4,PDGFRA,Grade
0,TCGA-DU-8164,0,51.30,white,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TCGA-QH-A6CY,0,38.72,white,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,TCGA-HW-A5KM,0,35.17,white,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TCGA-E1-A7YE,1,32.78,white,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,TCGA-S9-A6WG,0,31.51,white,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
df_info.head()

,Case_ID,Primary_Diagnosis
0,TCGA-DU-8164,"Oligodendroglioma, NOS"
1,TCGA-QH-A6CY,Mixed glioma
2,TCGA-HW-A5KM,"Astrocytoma, NOS"
3,TCGA-E1-A7YE,"Astrocytoma, anaplastic"
4,TCGA-S9-A6WG,"Astrocytoma, anaplastic"


In [99]:
# Dataframe structure
print("df_target no of rows:", df_target.shape[0], "and no of columns:", df_target.shape[1])
print("df_info no of rows:", df_info.shape[0], "and no of columns:", df_info.shape[1])

df_target no of rows: 839 and no of columns: 25
df_info no of rows: 862 and no of columns: 2


In [30]:
# Column Names
df_target.columns

Index(['Case_ID', 'Gender', 'Age_at_diagnosis', 'Race', 'IDH1', 'TP53', 'ATRX',
       'PTEN', 'EGFR', 'CIC', 'MUC16', 'PIK3CA', 'NF1', 'PIK3R1', 'FUBP1',
       'RB1', 'NOTCH1', 'BCOR', 'CSMD3', 'SMARCA4', 'GRIN2A', 'IDH2', 'FAT4',
       'PDGFRA', 'Grade'],
      dtype='object')

In [32]:
# Info stats
df_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839 entries, 0 to 838
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Case_ID           839 non-null    object 
 1   Gender            839 non-null    int64  
 2   Age_at_diagnosis  839 non-null    float64
 3   Race              839 non-null    object 
 4   IDH1              839 non-null    int64  
 5   TP53              839 non-null    int64  
 6   ATRX              839 non-null    int64  
 7   PTEN              839 non-null    int64  
 8   EGFR              839 non-null    int64  
 9   CIC               839 non-null    int64  
 10  MUC16             839 non-null    int64  
 11  PIK3CA            839 non-null    int64  
 12  NF1               839 non-null    int64  
 13  PIK3R1            839 non-null    int64  
 14  FUBP1             839 non-null    int64  
 15  RB1               839 non-null    int64  
 16  NOTCH1            839 non-null    int64  
 1

In [100]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862 entries, 0 to 861
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Case_ID            862 non-null    object
 1   Primary_Diagnosis  862 non-null    object
dtypes: object(2)
memory usage: 13.6+ KB


In [101]:
# Unique Values of df_target
df_target.nunique()

Case_ID             839
Gender                2
Age_at_diagnosis    766
Race                  4
IDH1                  2
TP53                  2
ATRX                  2
PTEN                  2
EGFR                  2
CIC                   2
MUC16                 2
PIK3CA                2
NF1                   2
PIK3R1                2
FUBP1                 2
RB1                   2
NOTCH1                2
BCOR                  2
CSMD3                 2
SMARCA4               2
GRIN2A                2
IDH2                  2
FAT4                  2
PDGFRA                2
Grade                 2
dtype: int64

In [39]:
df_target['Grade'].unique()

array([0, 1], dtype=int64)

In [40]:
df_target['EGFR'].unique()

array([0, 1], dtype=int64)

In [102]:
# Unique Values of df_target
df_info.nunique()

Case_ID              862
Primary_Diagnosis      7
dtype: int64

In [104]:
df_info['Primary_Diagnosis'].unique()

array(['Oligodendroglioma, NOS', 'Mixed glioma', 'Astrocytoma, NOS',
       'Astrocytoma, anaplastic', 'Oligodendroglioma, anaplastic', '--',
       'Glioblastoma'], dtype=object)

In [105]:
df_info['Primary_Diagnosis'].value_counts()

Primary_Diagnosis
Glioblastoma                     360
Astrocytoma, anaplastic          129
Mixed glioma                     128
Oligodendroglioma, NOS           108
Oligodendroglioma, anaplastic     75
Astrocytoma, NOS                  58
--                                 4
Name: count, dtype: int64

In [106]:
df_info[df_info['Primary_Diagnosis'] == "--"]

,Case_ID,Primary_Diagnosis
41,TCGA-R8-A6YH,--
671,TCGA-28-2501,--
706,TCGA-28-2510,--
794,TCGA-16-1048,--


# 4. Descriptive Statistics

In [33]:
df_target.describe()

,Gender,Age_at_diagnosis,IDH1,TP53,ATRX,PTEN,EGFR,CIC,MUC16,PIK3CA,...,RB1,NOTCH1,BCOR,CSMD3,SMARCA4,GRIN2A,IDH2,FAT4,PDGFRA,Grade
count,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000,...,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000,839.000000
mean,0.418355,50.935411,0.481526,0.414779,0.258641,0.168057,0.133492,0.132300,0.116806,0.087008,...,0.047676,0.045292,0.034565,0.032181,0.032181,0.032181,0.027414,0.027414,0.026222,0.419547
std,0.493583,15.702339,0.499957,0.492978,0.438149,0.374140,0.340309,0.339019,0.321380,0.282015,...,0.213206,0.208068,0.182784,0.176586,0.176586,0.176586,0.163383,0.163383,0.159889,0.493779
min,0.000000,14.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,38.055000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,51.550000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,62.800000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,89.290000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
df_target.describe(include='object')

,Case_ID,Race
count,839,839
unique,839,4
top,TCGA-DU-8164,white
freq,1,765


In [107]:
df_info.describe()

,Case_ID,Primary_Diagnosis
count,862,862
unique,862,7
top,TCGA-DU-8164,Glioblastoma
freq,1,360


# 4. Tabular Data Set-up

In [131]:
# TEMP DB in RAM
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [132]:
# PUSH Pandas DF --> TEMP DB
info_db = df_info.to_sql(name='Primary_Diagnosis', con=temp_db)

2024-03-11 22:09:04,609 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-11 22:09:04,609 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Primary_Diagnosis")
2024-03-11 22:09:04,610 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-11 22:09:04,610 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Primary_Diagnosis")
2024-03-11 22:09:04,611 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-11 22:09:04,612 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Primary_Diagnosis" (
	"index" BIGINT, 
	"Case_ID" TEXT, 
	"Primary_Diagnosis" TEXT
)


2024-03-11 22:09:04,612 INFO sqlalchemy.engine.Engine [no key 0.00045s] ()
2024-03-11 22:09:04,613 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Primary_Diagnosis_index" ON "Primary_Diagnosis" ("index")
2024-03-11 22:09:04,613 INFO sqlalchemy.engine.Engine [no key 0.00032s] ()
2024-03-11 22:09:04,617 INFO sqlalchemy.engine.Engine INSERT INTO "Primary_Diagnosis" ("index", "Case_ID", "Primary_Diagnosis") VALUES (?, ?, ?)
2024-03-11 22:09:04

In [133]:
# PUSH Pandas DF --> TEMP DB
target_db = df_target.to_sql(name='Target', con=temp_db)

2024-03-11 22:09:07,447 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-11 22:09:07,450 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Target")
2024-03-11 22:09:07,450 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-11 22:09:07,451 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Target")
2024-03-11 22:09:07,451 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-11 22:09:07,453 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Target" (
	"index" BIGINT, 
	"Case_ID" TEXT, 
	"Gender" BIGINT, 
	"Age_at_diagnosis" FLOAT, 
	"Race" TEXT, 
	"IDH1" BIGINT, 
	"TP53" BIGINT, 
	"ATRX" BIGINT, 
	"PTEN" BIGINT, 
	"EGFR" BIGINT, 
	"CIC" BIGINT, 
	"MUC16" BIGINT, 
	"PIK3CA" BIGINT, 
	"NF1" BIGINT, 
	"PIK3R1" BIGINT, 
	"FUBP1" BIGINT, 
	"RB1" BIGINT, 
	"NOTCH1" BIGINT, 
	"BCOR" BIGINT, 
	"CSMD3" BIGINT, 
	"SMARCA4" BIGINT, 
	"GRIN2A" BIGINT, 
	"IDH2" BIGINT, 
	"FAT4" BIGINT, 
	"PDGFRA" BIGINT, 
	"Grade" BIGINT
)


2024-03-11 22:09:07,453 INFO sqlalchemy.engine.Engine [no key 0.00038s]

In [138]:
# SQL query on TEMP DB
with temp_db.connect() as conn:
    # makes connection
    # run code indentation/block of code
    result = conn.execute(text("SELECT AVG(Age_at_diagnosis) FROM Target"))
    # auto close conn
    

2024-03-11 22:12:50,313 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-11 22:12:50,313 INFO sqlalchemy.engine.Engine SELECT AVG(Age_at_diagnosis) FROM Target
2024-03-11 22:12:50,314 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ()
2024-03-11 22:12:50,314 INFO sqlalchemy.engine.Engine ROLLBACK


In [139]:
result.all()

[(50.93541120381399,)]